
# Hand Gesture Controlled Car

This notebook explores using hand gestures detected through a camera stream to control a remote car. By utilizing a hand detection module and processing the video stream, specific hand gestures can correspond to car movements such as moving forward, backward, turning left, or right.
    

In [ ]:

import cv2
import numpy as np
import socket
import json
import time
import requests
from utils.HandTrackingModule import HandDetector
    


## Initializing the Hand Detector

We initialize the hand detector to recognize hands in the video stream, configuring it to detect a maximum of two hands with a certain detection confidence level.
    

In [ ]:

detectorH = HandDetector(maxHands=2, detectionCon=0.8)
    


## Configuring Streaming and Control Parameters

Set the stream URL for the car's camera and define the car's control interface (IP and port).
    

In [ ]:

stream_url = "http://192.168.4.1:81/stream"
IP = "192.168.4.1"
PORT = 100
    


## Sending Commands to the Car

A function to package and send control commands to the car via a TCP socket.
    

In [ ]:

def send_command(command):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((IP, PORT))
        sock.sendall(json.dumps(command).encode())
        time.sleep(0.5)
    


## Processing the Camera Stream

Extract frames from the camera stream, detect hands and interpret hand gestures to control the car's movements.
    

In [ ]:

session = requests.Session()
response = session.get(stream_url, stream=True)

if response.status_code == 200:
    bytes_data = bytes()
    while True:
        chunk = response.raw.read(1024)
        if not chunk:
            break
        bytes_data += chunk
        a = bytes_data.find(b'ÿØ')
        b = bytes_data.find(b'ÿÙ')
        while a != -1 and b != -1:
            jpg = bytes_data[a:b+2]
            bytes_data = bytes_data[b+2:]
            img = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
            
            if img is not None:
                img = detectorH.findHands(img)
                lmList, bbox = detectorH.findPosition(img, draw=True)
                if bbox:
                    fingers = detectorH.fingersUp()
                    # Gesture control logic

                cv2.imshow('Elegoo Smart Car Camera Stream', img)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            a = bytes_data.find(b'ÿØ')
            b = bytes_data.find(b'ÿÙ')

cv2.destroyAllWindows()
    


## Student Exercises

1. **Add More Gestures**: Extend the gesture recognition to include more hand gestures for additional car controls.

2. **Gesture-Based Speed Control**: Implement variable speed control based on the gesture's intensity or duration.

3. **Error Handling for Commands**: Improve the robustness of command sending with error handling and reconnection strategies.

4. **Visual Feedback**: Enhance the visual feedback in the video stream display for detected gestures and car responses.
    